In [1]:
from sklearn import metrics
import pandas as pd
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import ZZFeatureMap
from qiskit.providers.aer import AerSimulator
from qiskit.algorithms.optimizers import SPSA
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit_machine_learning.kernels.algorithms import QuantumKernelTrainer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [2]:
def read_data(csv_path):
    rdata = pd.read_csv(csv_path, skiprows=[1,1])
    split = int(len(rdata) * 0.50)

    X = rdata.drop(['Persons_killed', 'PROV_NAME', 'DIST_CODE', 'DIST_NAME', 'INC_DATE', 'Families_affected', 'Houses_destroyed'], axis = 1)
    y = rdata['Persons_killed']
    
    return X, y

In [3]:
# Read data and set training and testing sets
X, y  = read_data('./datasets/flash_flood_data.csv')

In [4]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   REGION                 720 non-null    object
 1   PROV_CODE              720 non-null    int64 
 2   INC_TYPE               720 non-null    object
 3   Persons_injured        720 non-null    int64 
 4   Individuals_affected2  720 non-null    int64 
 5   Houses_damaged         720 non-null    int64 
dtypes: int64(4), object(2)
memory usage: 33.9+ KB


In [5]:
X.head()

,REGION,PROV_CODE,INC_TYPE,Persons_injured,Individuals_affected2,Houses_damaged
0,Southern,33,Flood / flash flood,0,30,3
1,Southern,33,Flood / flash flood,0,25,3
2,Southern,33,Flood / flash flood,0,40,4
3,Southern,33,Flood / flash flood,0,232,29
4,Southern,32,Flood / flash flood,2,1360,129


In [6]:
pd.unique(X['INC_TYPE'])

array(['Flood / flash flood', 'Heavy snowfall', 'Earthquake',
       'Landslide / mudflow', 'Avalanche'], dtype=object)

In [7]:
pd.unique(X['REGION'])

array(['Southern', 'Eastern', 'South Eastern', 'Capital', 'Western',
       'North Eastern', 'Northern', 'Central Highland'], dtype=object)

In [8]:
pd.unique(X['PROV_CODE'])

array([33, 32, 13, 34, 26,  6,  7,  3, 24, 29, 30, 21, 15, 23, 16, 18, 20,
       17, 10,  4, 19, 28,  9, 27, 25,  5, 12,  2, 11, 14, 22, 31,  1],
      dtype=int64)

In [9]:
label_encoder = LabelEncoder()

# Encode labels in column
X['REGION']= label_encoder.fit_transform(X['REGION'])
X['INC_TYPE']= label_encoder.fit_transform(X['INC_TYPE'])

In [10]:
X.head()

,REGION,PROV_CODE,INC_TYPE,Persons_injured,Individuals_affected2,Houses_damaged
0,6,33,2,0,30,3
1,6,33,2,0,25,3
2,6,33,2,0,40,4
3,6,33,2,0,232,29
4,6,32,2,2,1360,129


In [11]:
y.head()

0    0
1    0
2    0
3    0
4    2
Name: Persons_killed, dtype: int64

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [13]:
# Setting up ZZFeatureMap circut with feature dimension the same number as number of features
feature_map = ZZFeatureMap(feature_dimension=X_train.shape[1])

# Use the statevector backend
backend = AerSimulator(method='statevector')

# Instantiate quantum kernel
quant_kernel = QuantumKernel(feature_map, quantum_instance=backend)

In [14]:
# Use QSVC for classification
qsvc = QSVC(quantum_kernel=quant_kernel)

# Fit the QSVC
qsvc.fit(X_train, y_train)

QSVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
     decision_function_shape='ovr', degree=3, gamma='scale', max_iter=-1,
     probability=False,
     quantum_kernel=<qiskit_machine_learning.kernels.quantum_kernel.QuantumKernel object at 0x0000017E7E030B88>,
     random_state=None, shrinking=True, tol=0.001, verbose=False)

In [15]:
# Predict the labels
labels_test = qsvc.predict(X_test)

In [16]:
# Evalaute the test accuracy
accuracy_test = metrics.balanced_accuracy_score(y_true=y_test, y_pred=labels_test)

print(f'accuracy test: {accuracy_test}')

accuracy test: 0.0625


In [18]:
num_features = X_train.shape[1]

# Number of parameters equal to number of features
num_params = num_features

In [21]:
# Create vector containing number of trainable parameters
user_params = ParameterVector('θ', num_params)

In [23]:
# Create circuit for num_features qbit system
fm0 = QuantumCircuit(num_features)

# First feature map component comprises a Y rotation with parameter theta on both qubit regs
for qubit in range(num_features):
    fm0.ry(user_params[(qubit%num_params)], qubit)

# Use ZZFeatureMap to represent input data
fm1 = ZZFeatureMap(feature_dimension=num_features)

# Compose both maps to create one feature map circuit
feature_map = fm0.compose(fm1)

# Use the statevector backend
backend = AerSimulator(method='statevector')

# Instantiate quantum kernel
quant_kernel = QuantumKernel(feature_map, user_parameters=user_params, quantum_instance=backend)

# Set up model optimizer
spsa_opt = SPSA(maxiter=10, learning_rate=0.05, perturbation=0.05)

# Instantiate a quantum kernel trainer
qkt = QuantumKernelTrainer(
    quantum_kernel=quant_kernel, loss="svc_loss", optimizer=spsa_opt, initial_point=[np.pi/2 for i in range(len(user_params))])

# Use QuantumKernelTrainer object to fit model to training data
qka_results = qkt.fit(X_train, y_train)
optimized_kernel = qka_results.quantum_kernel

# Use QSVC for classification
qsvc = QSVC(quantum_kernel=optimized_kernel)

# Fit the QSVC
qsvc.fit(X_train, y_train)

QSVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
     decision_function_shape='ovr', degree=3, gamma='scale', max_iter=-1,
     probability=False,
     quantum_kernel=<qiskit_machine_learning.kernels.quantum_kernel.QuantumKernel object at 0x0000017E7EFD5088>,
     random_state=None, shrinking=True, tol=0.001, verbose=False)

In [24]:
# Predict the labels
labels_test = qsvc.predict(X_test)

# Evalaute the test accuracy
accuracy_test = metrics.balanced_accuracy_score(y_true=y_test, y_pred=labels_test)
print(f'accuracy test: {accuracy_test}')

accuracy test: 0.0625
